In [2]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import transformers
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForTokenClassification
from torch import cuda

/home/kd/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [29]:
len(test.tokens[0])

753

In [4]:
train = pd.read_json('train.json')
test = pd.read_json('test.json')
sample_submission = pd.read_csv('sample_submission.csv')

In [5]:
train

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
...,...,...,...,...,...
6802,22678,EXAMPLE – JOURNEY MAP\n\nTHE CHALLENGE My w...,"[EXAMPLE, –, JOURNEY, MAP, \n\n, THE, CHALLENG...","[True, True, True, False, False, True, True, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6803,22679,Why Mind Mapping?\n\nMind maps are graphical r...,"[Why, Mind, Mapping, ?, \n\n, Mind, maps, are,...","[True, True, False, False, False, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6804,22681,"Challenge\n\nSo, a few months back, I had chos...","[Challenge, \n\n, So, ,, a, few, months, back,...","[False, False, False, True, True, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6805,22684,Brainstorming\n\nChallenge & Selection\n\nBrai...,"[Brainstorming, \n\n, Challenge, &, Selection,...","[False, False, True, True, False, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [6]:
MAX_LEN = 128#max([len(sentence) for sentence in sentences]) #3298
maxlen = MAX_LEN
TRAIN_BATCH_SIZE = 4 
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [7]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

#     sentence = sentence.strip()

#     for word, label in zip(sentence.split(), text_labels.split(",")):
    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

def augment_data_with_tokenizer(df):
    tokenized_sentences = []
    preserved_labels = []
    for i in range(len(df)):
        tokenized_sentence, preserved_label = tokenize_and_preserve_labels(df.tokens[i], df.labels[i], tokenizer)
        tokenized_sentences.append(tokenized_sentence)
        preserved_labels.append(preserved_label)
    
    df_aug = pd.DataFrame()
    df_aug['tokenized_sentences'] = tokenized_sentences
    df_aug['preserved_labels'] = preserved_labels
    return df_aug

def batchify_augmented_data(df_aug):
    
    df_aug_batched = pd.DataFrame()
    tokenized_sentences_batched = []
    preserved_labels_batched = []

    for i in range(len(df_aug)):
        tokens = df_aug['tokenized_sentences'][i]
        labels = df_aug['preserved_labels'][i]
        tokens = ["[CLS]"] + tokens + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        prev_j = 0
        if len(tokens) % maxlen == 0:
            upper_iter = len(tokens)
        else:
            upper_iter = (len(tokens) // maxlen + 2) * maxlen
        for j in range(maxlen, upper_iter, maxlen):
            if len(tokens[prev_j:j]) == maxlen:
                tokenized_sentences_batched.append(tokens[prev_j:j])
                preserved_labels_batched.append(labels[prev_j:j])
            else:
                tokenized_sentences_batched.append(tokens[prev_j:j] + ['[PAD]'for _ in range(maxlen - len(tokens[prev_j:j]))])
                preserved_labels_batched.append(labels[prev_j:j] + ["O" for _ in range(maxlen - len(labels[prev_j:j]))])

            prev_j = j

    df_aug_batched['tokenized_sentences'] = tokenized_sentences_batched
    df_aug_batched['preserved_labels'] = preserved_labels_batched
    return df_aug_batched
    


In [35]:
test

,document,full_text,tokens,trailing_whitespace
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal..."
5,86,Cheese Startup - Learning Launch ​by Eladio Am...,"[Cheese, Startup, -, Learning, Launch, ​by, El...","[True, True, True, True, True, True, True, Fal..."
6,93,Silvia Villalobos\n\nChallenge:\n\nThere is a ...,"[Silvia, Villalobos, \n\n, Challenge, :, \n\n,...","[True, False, False, False, False, False, True..."
7,104,Storytelling The Path to Innovation\n\nDr Sak...,"[Storytelling, , The, Path, to, Innovation, \...","[True, False, True, True, True, False, False, ..."
8,112,Reflection – Learning Launch\n\nFrancisco Ferr...,"[Reflection, –, Learning, Launch, \n\n, Franci...","[True, True, True, False, False, True, False, ..."
9,123,Gandhi Institute of Technology and Management ...,"[Gandhi, Institute, of, Technology, and, Manag...","[True, True, True, True, True, True, False, Tr..."


In [42]:
def tokenize_and_preserve_labels_test(sentence, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
#     labels = []

#     sentence = sentence.strip()

#     for word, label in zip(sentence.split(), text_labels.split(",")):
    for word in sentence:

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
#         labels.extend([label] * n_subwords)

    return tokenized_sentence #, labels

def augment_data_with_tokenizer_test(df):
    doc_ids = []
    tokenized_sentences = []
#     preserved_labels = []
    for i in range(len(df)):
        doc_ids.append(df.document[i])
        tokenized_sentence = tokenize_and_preserve_labels_test(df.tokens[i], tokenizer)
        tokenized_sentences.append(tokenized_sentence)
#         preserved_labels.append(preserved_label)
    
    df_aug = pd.DataFrame()
    df_aug['tokenized_sentences'] = tokenized_sentences
    df_aug['document'] = doc_ids
#     df_aug['preserved_labels'] = preserved_labels
    return df_aug

def batchify_augmented_data_test(df_aug):
    df_aug_batched = pd.DataFrame()
    doc_ids_batched = []
    tokenized_sentences_batched = []
#     preserved_labels_batched = []

    for i in range(len(df_aug)):
        tokens = df_aug['tokenized_sentences'][i]
        tokens = ["[CLS]"] + tokens + ["[SEP]"] # add special tokens
        prev_j = 0
        if len(tokens) % maxlen == 0:
            upper_iter = len(tokens)
        else:
            upper_iter = (len(tokens) // maxlen + 2) * maxlen
        for j in range(maxlen, upper_iter, maxlen):
            if len(tokens[prev_j:j]) == maxlen:
                tokenized_sentences_batched.append(tokens[prev_j:j])
            else:
                tokenized_sentences_batched.append(tokens[prev_j:j] + ['[PAD]'for _ in range(maxlen - len(tokens[prev_j:j]))])

            doc_ids_batched.append(df_aug['document'][i])

            prev_j = j

    df_aug_batched['tokenized_sentences'] = tokenized_sentences_batched
    df_aug_batched['document'] = doc_ids_batched
    return df_aug_batched
    

def reverse_tokenization(tokenized_sentence, preserved_label):
    """
    This function reverses the tokenization process. It takes in a
    tokenized sentence and a list of labels and returns a list of
    labels that corresponds to the original sentence.
    """

    original_sentence = []
    original_labels = []

    for token, label in zip(tokenized_sentence, preserved_label):
        # delete special tokens added during tokenization
        if token in ["[CLS]", "[SEP]"]:
            continue

        # If the token is part of a word piece, then just add the
        # label to the original labels list. Otherwise, add both
        # the token and the label to their respective lists.
        if token.startswith("##"):
            original_labels.append(label)
        else:
            original_sentence.append(token)
            original_labels.append(label)

    return original_sentence, original_labels

In [9]:
tokenized_sentences = []
preserved_labels = []
for i in range(len(train)):
    tokenized_sentence, preserved_label = tokenize_and_preserve_labels(train.tokens[i], train.labels[i], tokenizer)
    tokenized_sentences.append(tokenized_sentence)
    preserved_labels.append(preserved_label)
    
train_aug = pd.DataFrame()
train_aug['tokenized_sentences'] = tokenized_sentences
train_aug['preserved_labels'] = preserved_labels

In [10]:
train_aug_batched = pd.DataFrame()
tokenized_sentences_batched = []
preserved_labels_batched = []

for i in range(len(train_aug)):
    tokens = train_aug['tokenized_sentences'][i]
    labels = train_aug['preserved_labels'][i]
    tokens = ["[CLS]"] + tokens + ["[SEP]"] # add special tokens
    labels.insert(0, "O") # add outside label for [CLS] token
    labels.insert(-1, "O") # add outside label for [SEP] token
    
    prev_j = 0
    if len(tokens) % maxlen == 0:
        upper_iter = len(tokens)
    else:
        upper_iter = (len(tokens) // maxlen + 2) * maxlen
    for j in range(maxlen, upper_iter, maxlen):
        if len(tokens[prev_j:j]) == maxlen:
            tokenized_sentences_batched.append(tokens[prev_j:j])
            preserved_labels_batched.append(labels[prev_j:j])
        else:
            tokenized_sentences_batched.append(tokens[prev_j:j] + ['[PAD]'for _ in range(maxlen - len(tokens[prev_j:j]))])
            preserved_labels_batched.append(labels[prev_j:j] + ["O" for _ in range(maxlen - len(labels[prev_j:j]))])

        prev_j = j

train_aug_batched['tokenized_sentences'] = tokenized_sentences_batched
train_aug_batched['preserved_labels'] = preserved_labels_batched

In [11]:
sentences = train.tokens
tags = train.labels

# Vocabulary Building
# word_to_ix = {'<PAD>':0}
word_to_ix = {}
for sentence in sentences:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            
# tag_to_ix = {'<PAD>':0}
tag_to_ix = {}
for labels in tags:
    for tag in labels:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)
            
ix_to_tag = dict((v,k) for k,v in tag_to_ix.items())

In [12]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
#         sentence = self.data.sentence[index]  
        sentence = self.data.tokenized_sentences[index]  
#         word_labels = self.data.word_labels[index]  
        word_labels = self.data.preserved_labels[index]  
#         tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)
        tokenized_sentence, labels = sentence, word_labels
        
        # step 2: add special tokens (and corresponding labels)
#         tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
#         labels.insert(0, "O") # add outside label for [CLS] token
#         labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len
#         tokenized_sentences = []
#         if (len(tokenized_sentence) > maxlen):
#           # truncate
#             for i in range(0, len(tokenized_sentence)//maxlen+1, maxlen):
#                 if i == max_len
#                 tokenized_sentences.append(tokenized_sentence[:])
#                 labels = labels[:maxlen]
#         else:
#           # pad
#             tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
#             labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]
        
        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

#         label_ids = [label2id[label] for label in labels]
        label_ids = [tag_to_ix[label] for label in labels]

        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]
        
        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [13]:
train_size = 0.8
train_dataset = train_aug_batched.sample(frac=train_size,random_state=200)
test_dataset = train_aug_batched.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(train_aug_batched.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (41024, 2)
TRAIN Dataset: (32819, 2)
TEST Dataset: (8205, 2)


In [14]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [15]:
# model = BertForTokenClassification.from_pretrained('bert-base-uncased', 
#                                                    num_labels=len(ix_to_tag),
#                                                    id2label=ix_to_tag,
#                                                    label2id=tag_to_ix)

model = torch.load("bert_tuned.pt")
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [76]:
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

tensor(4.4799e-05, device='cuda:0', grad_fn=<NllLossBackward0>)

In [21]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [22]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_preds.extend(predictions)
        tr_labels.extend(targets)
        
        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [23]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 2.451850652694702
Training loss per 100 training steps: 0.23944752448524284
Training loss per 100 training steps: 0.129338664613173
Training loss per 100 training steps: 0.09043200793861723
Training loss per 100 training steps: 0.07079018756138751
Training loss per 100 training steps: 0.05973231376677625
Training loss per 100 training steps: 0.05161113163844501
Training loss per 100 training steps: 0.04618960790423817
Training loss per 100 training steps: 0.04128701465606071
Training loss per 100 training steps: 0.037359446822071284
Training loss per 100 training steps: 0.034697882481443634
Training loss per 100 training steps: 0.032183945825666894
Training loss per 100 training steps: 0.02970722712935813
Training loss per 100 training steps: 0.027890167407208996
Training loss per 100 training steps: 0.026503496906297316
Training loss per 100 training steps: 0.025148013693882063
Training loss per 100 training steps: 0.023844965511

In [24]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)
            
            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(targets)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
    
    #print(eval_labels)
    #print(eval_preds)

    labels = [ix_to_tag[id.item()] for id in eval_labels]
    predictions = [ix_to_tag[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [25]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 4.337540303822607e-05
Validation loss per 100 evaluation steps: 0.0005400009265418468
Validation loss per 100 evaluation steps: 0.0012391629308550179
Validation loss per 100 evaluation steps: 0.0013622688544588086
Validation loss per 100 evaluation steps: 0.0016894232468073374
Validation loss per 100 evaluation steps: 0.0015076147722932669
Validation loss per 100 evaluation steps: 0.0028990827613809365
Validation loss per 100 evaluation steps: 0.002884719690884073
Validation loss per 100 evaluation steps: 0.002855649315125866
Validation loss per 100 evaluation steps: 0.0030650649638324845
Validation loss per 100 evaluation steps: 0.0031567108345551607
Validation loss per 100 evaluation steps: 0.0034321594755636863
Validation loss per 100 evaluation steps: 0.0033397491947732695
Validation loss per 100 evaluation steps: 0.0033331800240040885
Validation loss per 100 evaluation steps: 0.0032000785024714636
Validation loss per 100 evaluation steps: 

In [26]:
# torch.save(model, "bert_tuned.pt")

In [71]:
def predict(model, sentence):
    inputs = tokenizer(sentence, is_split_into_words=True, padding='max_length', truncation=True, max_length=MAX_LEN, return_tensors="pt")
    # move to gpu
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    # forward pass
#     model.to(device)
    outputs = model(ids, mask)
    logits = outputs[0]
    active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [ix_to_tag[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    word_level_predictions = []
    for pair in wp_preds:
        if (pair[0].startswith(" ##")) or (pair[0] in ['[CLS]', '[SEP]', '[PAD]']):
            # skip prediction
            continue
        else:
            word_level_predictions.append(pair[1])

    # we join tokens, if they are not special ones
    str_rep = " ".join([t[0] for t in wp_preds if t[0] not in ['[CLS]', '[SEP]', '[PAD]']]).replace(" ##", "")
    print(str_rep)
    print(word_level_predictions)
    
    del ids
    del mask
    torch.cuda.empty_cache()
    
    return word_level_predictions

In [72]:
test_aug = augment_data_with_tokenizer_test(test)
test_aug_batched = batchify_augmented_data_test(test_aug)

In [77]:
word_level_predictions = predict(model, test_aug_batched.tokenized_sentences[1])

• it is accessible to all and does not require significant material investment and can be done quickly • it is scala # # ble • it allows cat # # ego # # rization and linking of information • it can be applied to any type of situation : note # # taking , problem solving , analysis , creation of new ideas • it is suitable for all people and is easy to learn • it is fun and encourages exchanges • it makes visible the dimension of projects , opportunities , inter # # con # # ne # # ctions • it synth # # es # # izes • it makes the project understand # # able
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

In [79]:
test_aug_batched['labels'] = np.empty((len(test_aug_batched), 0)).tolist()
for i in range(len(test_aug_batched)):
    word_level_predictions = predict(model, test_aug_batched.tokenized_sentences[i])
    test_aug_batched['labels'][i] = word_level_predictions

design thinking for innovation reflex # # ion - av # # ril 2021 - nat # # hal # # ie sy # # lla challenge & selection the tool i use to help all stakeholders finding their way through the complexity of a project is the mind map . what exactly is a mind map ? according to the definition of bu # # zan t . and bu # # zan b . ( 1999 , des # # sin # # e - moi l ' intelligence . paris : les editions d ' organisation . ) , the mind map ( or he # # uri # # stic diagram ) is a graphic
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

/tmp/ipykernel_4983/4208203702.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_aug_batched['labels'][i] = word_level_predictions
/tmp/ipykernel_4983/4208203702.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_aug_batched['labels'][i] = word_level_predictions
/tmp/ipykernel_4983/4208203702.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_aug_batched['labels'][i] = word_level_predictions
/tmp/ipykernel_4983/42082

he did not think for a moment that these modifications should have a low impact in case things do not go as well as expected . thanks to the application of the tool i was able to identify the key points of the company ' s needs and how each investor , according to their proposals , could help me , based on this i made the best decision . i chose to work with the second investor , we were able to supply the demand for the product , then the investment triple # # d and we were able to in # # ject it into research for product innovation , and make some small changes , which were being introduced to customers
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

/tmp/ipykernel_4983/4208203702.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_aug_batched['labels'][i] = word_level_predictions
/tmp/ipykernel_4983/4208203702.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_aug_batched['labels'][i] = word_level_predictions
/tmp/ipykernel_4983/4208203702.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_aug_batched['labels'][i] = word_level_predictions
/tmp/ipykernel_4983/42082

. as a student of design thinking , i can explore this tool from all corners and can confidently say that i can creative # # ly use it in number of inspiring ways at any appropriate phase of design thinking activities .
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
reflection – learning launch francisco ferreira challenge i take part of a social enter # # pre # # nu # # ers # # hip group in my university . we were in contact with cap # # ao das ant # # as , a rural community in the suburbs of sao carlos ( sp - brazil ) . we had the intention to cr # # iate , with the local producers , a project that wo # # ld improve the quality of life in the neighborhood . for that , in the first two months we begin to listen each farmer of cap # # ao and try identify the principal issues of the place and their resident
['O', 'O

/tmp/ipykernel_4983/4208203702.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_aug_batched['labels'][i] = word_level_predictions
/tmp/ipykernel_4983/4208203702.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_aug_batched['labels'][i] = word_level_predictions
/tmp/ipykernel_4983/4208203702.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_aug_batched['labels'][i] = word_level_predictions
/tmp/ipykernel_4983/42082

re # # mun # # eration or alt # # ru # # istic punishment ( stigma # # ti # # zation ) . taking all things into consideration , we can conclude , that alt # # ru # # ism has a significant economic effect not only on one particular person or a small social group , but on the whole society . and strange , as it may seem , alt # # ru # # istic behavior can be profitable for the alt # # ru # # ist as well as for someone , who receives help . moreover , many social organizations , even economic ones , would not work at all
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

/tmp/ipykernel_4983/4208203702.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_aug_batched['labels'][i] = word_level_predictions
/tmp/ipykernel_4983/4208203702.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_aug_batched['labels'][i] = word_level_predictions
/tmp/ipykernel_4983/4208203702.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_aug_batched['labels'][i] = word_level_predictions
/tmp/ipykernel_4983/42082

In [80]:
test_aug_batched

,tokenized_sentences,document,labels
0,"[[CLS], design, thinking, for, innovation, ref...",7,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"[•, it, is, accessible, to, all, and, does, no...",7,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[point, generates, ideas, /, work, areas, ,, i...",7,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[topic, to, be, addressed, ., in, the, type, o...",7,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[., this, second, workshop, also, lasts, two, ...",7,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...,...
61,"[re, ##mun, ##eration, or, alt, ##ru, ##istic,...",123,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
62,"[system, ., 3, kazan, ##ts, ##eva, t, ., v, .,...",123,"[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, B-..."
63,"[2018, /, 36, ##39, ##8, -, et, ##iche, ##sko,...",123,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
64,"[k, -, sp, ##rave, ##dl, ##ivo, ##mu, -, so, #...",123,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [81]:
sample_submission

,row_id,document,token,label
0,0,7,9,B-NAME_STUDENT
1,1,7,10,I-NAME_STUDENT
2,2,7,482,B-NAME_STUDENT
3,3,7,483,I-NAME_STUDENT
4,4,7,741,B-NAME_STUDENT
5,5,7,742,I-NAME_STUDENT
6,6,10,0,B-NAME_STUDENT
7,7,10,1,I-NAME_STUDENT
8,8,10,464,B-NAME_STUDENT
9,9,10,465,I-NAME_STUDENT


In [87]:
result = pd.DataFrame()
result['document'] = []
result['tokens'] = []
result['labels'] = []

for doc in test_aug_batched.document.unique():
    df_this_doc = test_aug_batched[test_aug_batched.document == doc]
    preserved_labels_this_doc = []
    preserved_tokens_this_doc = []
    for i in range(len(df_this_doc)):
        labels = df_this_doc['labels'][i]
        tokens = df_this_doc['tokenized_sentences'][i]
        original_tokens, original_labels = reverse_tokenization(tokens, labels)
        preserved_labels_this_doc.extend(original_labels)
        preserved_tokens_this_doc.extend(original_tokens)
    print(type(result))
    result = result.append({'document': doc, 'tokens': preserved_tokens_this_doc, 'labels': preserved_labels_this_doc}, ignore_index=True)
    # result.append({'document': doc, 'tokens': preserved_tokens_this_doc, 'labels': preserved_labels_this_doc}, ignore_index=True)


<class 'pandas.core.frame.DataFrame'>


AttributeError: 'DataFrame' object has no attribute 'append'

In [88]:
rows_list = []

for doc in test_aug_batched.document.unique():
    df_this_doc = test_aug_batched[test_aug_batched.document == doc]
    preserved_labels_this_doc = []
    preserved_tokens_this_doc = []
    for i in range(len(df_this_doc)):
        labels = df_this_doc['labels'].iloc[i]
        tokens = df_this_doc['tokenized_sentences'].iloc[i]
        original_tokens, original_labels = reverse_tokenization(tokens, labels)
        preserved_labels_this_doc.extend(original_labels)
        preserved_tokens_this_doc.extend(original_tokens)
    dict1 = {'document': doc, 'tokens': preserved_tokens_this_doc, 'labels': preserved_labels_this_doc}
    rows_list.append(dict1)

result = pd.DataFrame(rows_list)

In [89]:
result

,document,tokens,labels
0,7,"[design, thinking, for, innovation, reflex, -,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,10,"[diego, estrada, design, thinking, assignment,...","[I-NAME_STUDENT, O, O, O, O, O, O, O, O, O, O,..."
2,16,"[reporting, process, by, gilbert, ga, challeng...","[O, O, B-NAME_STUDENT, O, O, O, O, O, O, O, O,..."
3,20,"[design, thinking, for, innovation, sin, sam, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,56,"[assignment, :, visual, reflection, submitted,...","[O, O, O, O, O, O, O, O, O, O, B-NAME_STUDENT,..."
5,86,"[cheese, startup, -, learning, launch, by, el,...","[O, O, O, O, O, B-NAME_STUDENT, O, O, B-NAME_S..."
6,93,"[silvia, villa, challenge, :, there, is, a, co...","[I-NAME_STUDENT, O, O, O, O, O, O, O, O, O, O,..."
7,104,"[storytelling, the, path, to, innovation, dr, ...","[O, O, O, O, O, O, O, O, B-NAME_STUDENT, B-NAM..."
8,112,"[reflection, –, learning, launch, francisco, f...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
9,123,"[gandhi, institute, of, technology, and, manag...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
